# Аналитические и численные методы линейной регрессии

###  В этом ноутбуке рассматриваются различные подходы к решению задачи **линейной регрессии** с использованием **L2-потери** (квадратичной ошибки) $$ \min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2 $$
где:
- $\mathbf{X}$ — матрица признаков,
- $\mathbf{w}$ — вектор весов (параметров модели),
- $\mathbf{y}$ — вектор целевой переменной.

Эта постановка известна как **метод наименьших квадратов (OLS)**. Одно из её ключевых преимуществ — существование **АНАЛИТИЧЕСКОГО решения в замкнутой форме**:
$$
\mathbf{w} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y},
$$
при условии, что матрица $\mathbf{X}^T\mathbf{X}$ обратима. Это позволяет находить оптимальные веса без итераций — как в случае одного признака, так и нескольких.

Мы сравниваем:
- Аналитические (непараметрические) решения вручную,
- Реализацию `LinearRegression` из `scikit-learn`, которая использует SVD-разложение для численно устойчивого решения.

### Почему мы не рассматриваем минимизацию $ \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_1 $?

В этом ноутбуке мы фокусируемся на задаче линейной регрессии с **L2-потерей** (метод наименьших квадратов),
для которой существуют **аналитические решения** — как в случае одного признака, так и в многомерном случае. 
Однако, если вместо L2 использовать **L1-норму**:
$$
\min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_1,
$$
— **аналитического решения в замкнутой форме не существует**. Причина в том, что L1-норма не является гладкой функцией (её производная не определена при нулевых остатках), и стандартные методы математического анализа для поиска минимума через приравнивание градиента к нулю неприменимы.

- **Задача минимизации L1-потери** (т.е. $\min \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_1$) на практике решается следующими способами:

  1. **"Из коробки"**:
     - `QuantileRegressor(quantile=0.5)` — в `scikit-learn`, эквивалентна L1-регрессии.
     - `LADRegressor` — начиная с `scikit-learn >= 1.3`, прямая реализация.
     Оба метода используют **линейное программирование** под капотом (например, солвер `highs`).
     Как L1-регрессия сводится к задаче линейного программирования (LP)?

     Задача минимизации L1-потери $ \min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_1 $ не имеет аналитического              решения, но может быть **точно решена численно**, так как её можно **переформулировать как задачу линейного          программирования (Linear Programming, LP)**. Переформулировка задачи:

     Вводятся вспомогательные переменные $\mathbf{r}^+ \in \mathbb{R}^n$, $\mathbf{r}^- \in \mathbb{R}^n$ — положительные и отрицательные части остатков.

    Каждый остаток $x_i^T \mathbf{w} - y_i$ можно представить как $ x_i^T \mathbf{w} - y_i = r_i^+ - r_i^-, \quad \text{где } r_i^+ \geq 0, \; r_i^- \geq 0 $. Тогда модуль остатка $|x_i^T \mathbf{w} - y_i| = r_i^+ + r_i^- $

    Формальная постановка LP-задачи $\min_{\mathbf{w},\, \mathbf{r}^+,\, \mathbf{r}^-} \sum_{i=1}^n (r_i^+ + r_i^-)$ при ограничениях $\mathbf{X}\mathbf{w} - \mathbf{y} = \mathbf{r}^+ - \mathbf{r}^-$,  $\mathbf{r}^+ \geq 0^-, \quad \mathbf{r}^- \geq 0 $

    Целевая функция и ограничения линейные, это классическая задача **линейного программирования**.
    
    
  2. **Через `scipy.optimize.linprog`**:
     - Можно сформулировать задачу как LP вручную.
     - Точно, но требует ручного построения матриц ограничений.
     - Подходит для малых задач.

  3. **Субградиентный спуск вручную**:
     - Реализуется с использованием `np.sign(остатки)`.
     - Просто, но медленно и требует подбора шага.
     - Нет в `SGDRegressor` — нужно писать самому.

- **Добавление L1 или L2-регуляризации к L1-потере**:
  - Теоретически возможно:  
    $
    \min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_1 + \alpha_1 \|\mathbf{w}\|_1 + \alpha_2 \|\mathbf{w}\|_2^2
    $
  - **Но на практике**:
    - Не реализовано в `scikit-learn`,
    - Не используется массово,
    - Реализуется **вручную** через библиотеки вроде `cvxpy`.
 
> Итак, для L1-потери **лучше использовать `QuantileRegressor` или `LADRegressor` из коробки**.  
> Всё остальное — либо ручная реализация, либо специализированные инструменты вроде `cvxpy`, которые нужны только в особых случаях.

In [2]:
import numpy as np
import pandas as pd

# Датасет: 10 наблюдений, один признак
data_single = {
    'x': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'y': [3.5, 6.8, 9.2, 12.0, 15.1, 18.3, 21.0, 24.2, 27.1, 30.5]
}

df_single = pd.DataFrame(data_single)
print("Датасет (один признак):")
print(df_single)

Датасет (один признак):
    x     y
0   1   3.5
1   2   6.8
2   3   9.2
3   4  12.0
4   5  15.1
5   6  18.3
6   7  21.0
7   8  24.2
8   9  27.1
9  10  30.5


## 1. Аналитическое решение парной линейной регрессии

Для случая **ОДНОГО** признака $ x $ и целевой переменной $ y $ можно найти оптимальные параметры модели $ y = mx + b $ **аналитически**, без итераций.

###  Цель
Найти такие $ m $ (наклон) и $ b $ (свободный член), которые минимизируют сумму квадратов ошибок (MSE).

---

###  Формулы МНК (метод наименьших квадратов)

Для $ n $ наблюдений:

- **Наклон $ m $:**
  $$
  m = \frac{n \sum (x_i y_i) - \sum x_i \sum y_i}{n \sum (x_i^2) - (\sum x_i)^2}
  $$

- **Свободный член $ b $:**
  $$
  b = \bar{y} - m \bar{x}
  $$
  где $ \bar{x} $ и $ \bar{y} $ — средние значения.

Эта формула — частный случай метода наименьших квадратов (МНК), выведенный только для случая одного признака x .
Она не работает при двух и более признаках, потому что: 

- теряется возможность учитывать взаимное влияние признаков,
- нельзя выразить несколько коэффициентов в одной скалярной формуле.


###  Реализация формулы в коде:

In [3]:
points = list(df_single.itertuples(index=False))

n = len(points)

# Вычисляем m (наклон)
m = (n * sum(p.x * p.y for p in points) - sum(p.x for p in points) * sum(p.y for p in points)) / \
    (n * sum(p.x**2 for p in points) - (sum(p.x for p in points))**2)

# Вычисляем b (свободный член)
b = (sum(p.y for p in points) / n) - m * (sum(p.x for p in points) / n)

print(f"\nМетод 1 (ручная формула):")
print(f"y = {m:.3f}x + {b:.3f}")


Метод 1 (ручная формула):
y = 2.972x + 0.427


##  2. Аналитическое (матричное) решение парной и множественной линейной регрессии (МНК)

Этот метод позволяет найти оптимальные параметры модели \( y = mx + b \) аналитически, используя линейную алгебру. Он работает не только для одного признака, но и для любого числа признаков — это универсальный подход, основанный на методе наименьших квадратов (МНК).

### Математическая формула

Оптимальные параметры $ \mathbf{w} $ находятся по формуле метода наименьших квадратов:

$$
\mathbf{w} = (X^T X)^{-1} X^T \mathbf{y}
$$

Где:
- $ X $ — матрица признаков с добавленным столбцом единиц (для свободного члена $ b $)
- $ \mathbf{y} $ — вектор целевых значений
- $ \mathbf{w} = [b, m] $ — вектор параметров модели


Хотя в теории решение выражается формулой, на практике в библиотеках, включая `scikit-learn`, она **не используется напрямую** из-за численной неустойчивости и может приводить к значительным ошибкам в присутствии мультиколлинеарности, шума или при большом количестве признаков.

### Реализация формулы в коде:

In [4]:
X = np.column_stack([np.ones(n), df_single['x']])  # [1, x]
y = df_single['y'].values

# Аналитическое решение: w = (X^T X)^{-1} X^T y
w = np.linalg.inv(X.T @ X) @ X.T @ y

b_mnk, m_mnk = w[0], w[1]

print(f"\nМетод 2 (матричная формула):")
print(f"y = {m_mnk:.3f}x + {b_mnk:.3f}")


Метод 2 (матричная формула):
y = 2.972x + 0.427


## 3. Метод `LinearRegression` из `scikit-learn`

Внутри `LinearRegression` используется функция `scipy.linalg.lstsq` (или аналог из `numpy`), которая решает задачу наименьших квадратов **с помощью SVD** — сингулярного разложения.

Матрица признаков $\mathbf{X}$ разлагается как:
$
\mathbf{X} = U \Sigma V^T
$
где:
- $U$ и $V$ — ортогональные матрицы,
- $\Sigma$ — диагональная матрица сингулярных значений.

Реальные данные почти всегда содержат шум, мультиколлинеарность, избыток признаков, пропущенные или коррелирующие переменные, что делает матрицу X плохо обусловленной или даже вырожденной, поэтому под капотом вызывается SVD с обрезанием малых сингулярных значений, и решение вычисляется через псевдообратную матрицу Мура-Пенроуза

Вектор весов вычисляется как:
$
\mathbf{w} = \mathbf{X}^+ \mathbf{y} = V \Sigma^+ U^T \mathbf{y}
$
где $\mathbf{X}^+$ — **матрица Мура – Пенроуза** (псевдообратная матрица), а $\Sigma^+$ — обобщённая обратная к $\Sigma$:
- $(\Sigma^+)_{ii} = 1 / \sigma_i$, если $\sigma_i > 0$,
- $(\Sigma^+)_{ii} = 0$, если $\sigma_i = 0$.

> Это решение:
> - Всегда существует,
> - Имеет минимальную норму среди всех возможных решений,
> - Устойчиво к мультиколлинеарности и вырожденности,
> - Работает даже при $p > n$ (больше признаков, чем объектов).

Существует еще и частный случай $\mathbf{w} = V \Sigma^{-1} U^T \mathbf{y}$ (с обычным $\Sigma^{-1}$), но работает только при полном ранге $\mathbf{X}$, а **на практике используется обычно обобщённая версия с $\Sigma^+$** — она универсальна и численно устойчива.

---

### Почему не используется формула $(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$?

Несмотря на математическую привлекательность аналитической формулы, её прямое применение в реальных данных проблематично:

| Проблема                   | Описание |
|----------------------------|--------|
| **Численная неустойчивость** | При мультиколлинеарности $\mathbf{X}^T\mathbf{X}$ становится плохо обусловленной, и обращение приводит к большим ошибкам. |
| **Потеря точности**         | Вычисление $\mathbf{X}^T\mathbf{X}$ удваивает ошибку округления. |
| **Вырожденность**           | Если $p > n$ или признаки линейно зависимы, $\mathbf{X}^T\mathbf{X}$ необратима. |
| **Риск переполнения**       | При больших значениях в $\mathbf{X}$ произведение $\mathbf{X}^T\mathbf{X}$ может привести к переполнению. |

---

### Преимущества SVD в `LinearRegression`

- Высокая численная устойчивость,
- Работа с вырожденными и плохо обусловленными матрицами,
- Поддержка $p > n$,
- Контроль устойчивости через параметр `rcond` (порог для малых сингулярных значений).

Использование SVD и псевдообратной матрицы делает `LinearRegression` из `scikit-learn` **надёжным и универсальным** даже на "грязных" или сложных по структуре данных.

### Методы решения задачи линейной регрессии в `scikit-learn`

В данном исследовании мы используем **SVD-решение** (`LinearRegression`), но в `scikit-learn` также доступны другие аналитические методы для решения задачи:

$
\min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2 + \alpha \|\mathbf{w}\|_2^2
$

#### 1. Основные подходы

- **`LinearRegression`**  
  - **Алгоритм**: SVD или QR-разложение  
  - **Формула решения**:  
   $ 
    \mathbf{w} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}
    \$ 
    
  - **Особенности**:  
    - Нет регуляризации ($\alpha = 0$)  
    - Устойчив к вырожденным или плохо обусловленным матрицам за счёт SVD  
    - Работает даже при линейной зависимости признаков  

- **`Ridge` с `solver='svd'`**  
  - **Формула**:  
    $
    \mathbf{w} = (\mathbf{X}^T\mathbf{X} + \alpha \mathbf{I})^{-1}\mathbf{X}^T\mathbf{y}
    $ 
  - **Отличие**: добавлена **L2-регуляризация** (параметр `alpha > 0`)  
  - Улучшает устойчивость при мультиколлинеарности  

#### 2. Альтернативные аналитические методы

- **`Ridge` с `solver='cholesky'`**  
  - Использует разложение Холецкого для $\mathbf{X}^T\mathbf{X} + \alpha \mathbf{I}$  
  - **Плюсы**: высокая скорость при малом числе признаков  
  - **Минусы**: требует положительной определённости матрицы (регуляризация обязательна)  

- **`Ridge` с `solver='lsqr'`**  
  - Использует итерационный метод на основе QR-разложения  
  - Поддерживает разреженные матрицы  
  - Эффективен при большом числе признаков или наблюдений  

#### 3. Сравнительная таблица методов

| Метод                     | Алгоритм          | Регуляризация | Устойчивость       | Скорость     |
|---------------------------|-------------------|---------------|--------------------|--------------|
| `LinearRegression`        | SVD / QR          | Нет           | Высокая            | Средняя      |
| `Ridge(solver='svd')`     | SVD               | L2            | Очень высокая      | Медленная    |
| `Ridge(solver='cholesky')`| Холецкого         | L2            | Средняя (с $\alpha$)| Быстрая      |
| `Ridge(solver='lsqr')`    | Итеративный QR    | L2            | Высокая            | Очень быстрая|

> Все перечисленные методы:
> - Являются **аналитическими** (или основаны на прямых матричных разложениях)  
> - Решают обобщённую задачу минимизации:  
>   $
   \min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2 + \alpha \|\mathbf{w}\|_2^2
   $
> - **Не требуют подбора шага обучения** или числа итераций  
> - Отличаются по устойчивости, скорости и требованиям к данным

### Неаналитические методы для задачи L2-потери

Хотя для задачи минимизации L2-потери  
$
\min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2
$
существуют аналитические решения (например, через SVD), при больших объёмах данных они становятся вычислительно затратными.  
В таких случаях используются **итерационные (неаналитические) методы**, не требующие обращения матриц: Gradient Descent, Stochastic Gradient Descent (SGD), Mini-batch Gradient Descent, SAG, SAGA, LSQR, Coordinate Descent, ADMM.

##  Методы решения задачи линейной регрессии с L2-регуляризацией: аналитические и численные подходы


| Метод | Алгоритм | Регуляризация | Подходит для больших данных | Библиотеки и реализация | Особенности | Когда использовать |
|-------|---------|---------------|-------------------------------|--------------------------|-------------|----------------------|
| LinearRegression | SVD или QR-разложение | Нет | Нет (при \( n, p > 10^4 \) медленно) | `scikit-learn`: `LinearRegression` | Точный, устойчивый, но не масштабируемый | Когда данные небольшие, нет переобучения, и нужна максимальная точность без регуляризации |
| Ridge (solver='svd') | SVD разложение | L2 | Нет | `scikit-learn`: `Ridge(solver='svd')` | Точный, работает при коллинеарности, медленно на больших данных | Для малых данных с мультиколлинеарностью, когда важна точность и интерпретируемость |
| Ridge (solver='cholesky') | Разложение Холецкого | L2 | Средне | `scikit-learn`: `Ridge(solver='cholesky')` | Быстро при малом числе признаков, требует \( \alpha > 0 \) | Когда \( n, p < 10^4 \), данные плотные, нужна быстрая сходимость |
| Ridge (solver='lsqr') | Итерационный QR | L2 | Да | `scikit-learn`: `Ridge(solver='lsqr')` | Поддерживает разреженные матрицы, средняя скорость, рекомендуется для больших данных | Универсальный выбор для средних и больших данных, особенно с разреженными признаками |
| Ridge (solver='sag') | Стохастический усреднённый градиент (SAG) | L2 | Да (при \( n, p > 10^4 \)) | `scikit-learn`: `Ridge(solver='sag')`; альтернативы: `river` | Быстрая сходимость, требует масштабирования признаков | Для больших, но умеренных по размеру выборок (\( n > 10^4 \)), если признаки стандартизованы |
| Ridge (solver='saga') | Улучшенный SAG (SAGA) | L2 | Да | `scikit-learn`: `Ridge(solver='saga')`; альтернативы: `river`, `pytorch-optimizer` | Поддерживает L2; более устойчив, чем SAG, подходит для больших выборок | Аналог SAG, но лучше ведёт себя при сложных данных; хорош для больших и разреженных данных |
| SGDRegressor | Стохастический градиентный спуск (SGD) | L2 (опционально) | Да (очень большие выборки) | `scikit-learn`: `SGDRegressor(loss='squared_error', penalty='l2')`; альтернативы: `river`, `PyTorch`, `TensorFlow` | Высокая гибкость, требует подбора шага обучения и масштабирования признаков, подходит для онлайн-обучения | Когда данные очень большие (не помещаются в память), потоковые, или нужно онлайн-обучение |


Перечисленные методы охватывают **разные подходы к решению задачи линейной регрессии с L2-регуляризацией**, и их выбор зависит от размера данных, требований к скорости, точности и масштабируемости.

- **Аналитические методы** (`LinearRegression`, `Ridge` с `svd`, `cholesky`, `lsqr`) дают точное решение и **не требуют масштабирования признаков**, но **не масштабируются** на очень большие выборки.
- **Градиентные методы** (`Ridge` с `sag`, `saga`, `SGDRegressor`) являются **итерационными**, **масштабируемыми** и подходят для больших данных, но **требуют стандартизации признаков** и тщательного подбора гиперпараметров.
- `SGDRegressor` — **наиболее гибкий** метод: поддерживает онлайн-обучение, потоковые данные и различные типы регуляризации, но требует настройки шага обучения.
- Для **разреженных данных** рекомендуются `Ridge(solver='lsqr')`, `'sag'`, `'saga'` или `SGDRegressor`.
- При **малом числе наблюдений** и необходимости точного решения лучше использовать `LinearRegression` или `Ridge` с `svd`/`cholesky`.

Таким образом, выбор метода должен основываться на балансе между **точностью**, **скоростью**, **объёмом данных** и **практическими ограничениями** (например, память, онлайн-обучение).

---

### L1-регуляризация (Lasso)

L1-регуляризация добавляется к L2-потере для получения разреженных решений:
$
\min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2 + \alpha \|\mathbf{w}\|_1
$

Эта задача известна как **Lasso-регрессия**.  
Она не имеет аналитического решения из-за недифференцируемости L1-нормы $\|\mathbf{w}\|_1$, и решается численно (обычно — координатным спуском).

**Основная цель L1-регуляризации** — **отбор признаков**: обнуление малозначимых весов, что упрощает интерпретацию модели.  
Особенно полезна, когда:
- Число признаков велико ($p \gg n$),
- Многие признаки слабо влияют на целевую переменную.

Однако Lasso склонен:
- Обнулять один признак из группы коррелирующих,
- Зависеть от масштаба признаков (требует стандартизации).

#### Методы решения задачи Lasso (L2-ошибка + L1-регуляризация)

| Метод | Основная идея | Скорость и устойчивость | Подходит для больших данных | Поддержка разреженности | Библиотеки и реализация | Особенности | Когда использовать | Ключевое различие с альтернативой |
|-------|---------------|--------------------------|-------------------------------|--------------------------|----------------------------|-------------|----------------------|-------------------------------|
| Coordinate Descent | Поочерёдная оптимизация одного веса при фиксированных остальных с учётом L1-штрафа | Высокая скорость, хорошая устойчивость | Да (при умеренном числе признаков) | Отлично | `scikit-learn`: `Lasso`, `LassoCV` | Эффективен и стабилен; используется по умолчанию | Для задач с отбором признаков, когда требуется разреженная модель | В отличие от ElasticNet, даёт **максимально разреженное решение**; не учитывает корреляцию между признаками |
| ISTA / Proximal Gradient | Градиентный шаг + soft thresholding для L1-нормы | Средняя скорость, хорошая устойчивость | Да | Да | `scikit-learn`: нет прямого класса; альтернативы: `pylops`, `sporco`, ручная реализация | Базовый проксимальный метод; лежит в основе многих решателей | В исследованиях и кастомных моделях с L1-регуляризацией | Проксимальный оператор проще, чем в ElasticNet: **не включает L2-градиент** |
| FISTA | Ускоренная версия ISTA (аналог метода Нестерова) | Высокая скорость, хорошая устойчивость | Да | Да | `scikit-learn`: нет встроенного класса; альтернативы: `pylops.FISTA`, `sporco`, `autograd`, `JAX` | Ускоренная сходимость; популярен в итерационной оптимизации | При разработке эффективных решателей, особенно в онлайн-режиме | Не включает L2-коррекцию в шаге, что делает его **менее устойчивым при мультиколлинеарности**, чем в ElasticNet |
| LARS (Least Angle Regression) | Геометрический метод построения траектории коэффициентов с учётом корреляций | Средняя скорость, отличная устойчивость | Нет | Умеренная | `scikit-learn`: `LassoLars`, `LassoLarsCV`, `lars_path()` | Позволяет построить полный путь коэффициентов при росте \( \alpha \) | Для анализа отбора признаков и визуализации "Lasso path" | **Уникален для Lasso**: в ElasticNet нет аналога, так как L1+L2 нарушает линейность пути |
| ADMM (Alternating Direction Method of Multipliers) | Разделение задачи на подзадачи с двойственными переменными | Средняя скорость, отличная устойчивость | Да (в распределённых системах) | Да | `scikit-learn`: нет; альтернативы: `scikit-learn-extra`, `pyADMM`, `OSQP`, `CVXPY` | Масштабируемый; подходит для сложных ограничений | В промышленных системах и распределённых вычислениях | В Lasso — проще: **один проксимальный оператор (L1)**; в ElasticNet — сложнее из-за смешанного штрафа |
| Subgradient Method | Использование субградиента из-за негладкости L1-нормы | Низкая скорость, удовлетворительная устойчивость | Да (но медленно) | Да | `scikit-learn`: нет реализации; альтернативы: ручная реализация на `NumPy`, `PyTorch`, `TensorFlow` | Прост в реализации, но неэффективен | Только для учебных целей или базового понимания | В Lasso — субградиент только от L1; в ElasticNet — сумма субградиента L1 и градиента L2 |





### Смешанная регуляризация (ElasticNet)

Смешанная L1+L2-регуляризация объединяет преимущества Ridge и Lasso:
$
\min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|_2^2 + \alpha \left( \rho \|\mathbf{w}\|_1 + (1 - \rho) \|\mathbf{w}\|_2^2 \right)
$

где:
- $\alpha$ — общий коэффициент регуляризации,
- $\rho \in [0, 1]$ — доля L1-регуляризации.

Эта модель называется **ElasticNet**.

**Преимущества перед Lasso**:
- При наличии **групп коррелирующих признаков**, ElasticNet склонен сохранять их все (вместо выбора одного, как Lasso),
- Более устойчива к шуму и переобучению,
- Гибко балансирует между разреженностью (L1) и устойчивостью (L2).

**Недостатки**:
- Два гиперпараметра ($\alpha$, $\rho$), что усложняет подбор,
- Требует масштабирования признаков,
- Решается только итерационно.

####  Методы решения задачи ElasticNet (L2-ошибка + L1 + L2-регуляризация)

| Метод | Основная идея | Скорость и устойчивость | Подходит для больших данных | Поддержка разреженности | Библиотеки и реализация | Особенности | Когда использовать | Ключевое различие с альтернативой |
|-------|---------------|--------------------------|-------------------------------|--------------------------|----------------------------|-------------|----------------------|-------------------------------|
| Coordinate Descent | Поочерёдная оптимизация с учётом комбинированного штрафа \( \rho\|w\|_1 + (1-\rho)\|w\|_2^2 \) | Высокая скорость, хорошая устойчивость | Да (при умеренном числе признаков) | Хорошая (менее разреженная, чем у Lasso) | `scikit-learn`: `ElasticNet`, `ElasticNetCV` | Учитывает баланс между L1 и L2 через параметр \( \rho \); устойчив при групповой корреляции | Когда признаки коррелируют группами, и нужен компромисс между отбором и устойчивостью | В отличие от Lasso, **стабилен при мультиколлинеарности**: L2-штраф предотвращает "случайный" отбор одного из группы коррелирующих признаков |
| Proximal Gradient (ISTA) | Градиентный шаг + soft thresholding с поправкой на L2-компоненту | Средняя скорость, хорошая устойчивость | Да | Да | `scikit-learn`: нет прямого класса; альтернативы: `pylops`, `sporco`, ручная реализация | Требует модификации prox-оператора для смешанного штрафа | В исследованиях с составными регуляризаторами | Проксимальный оператор включает **оба штрафа**: soft thresholding для L1 и градиентное смещение от L2 |
| FISTA | Ускоренная версия proximal gradient для смешанной регуляризации | Высокая скорость, хорошая устойчивость | Да | Да | `scikit-learn`: нет встроенного класса; альтернативы: `pylops.FISTA`, `sporco`, `autograd`, `JAX` | Ускоренная сходимость; эффективен при большом числе итераций | В кастомных моделях с быстрой сходимостью | Благодаря L2-штрафу — **более стабильная сходимость** по сравнению с Lasso при коррелирующих признаках |
| ADMM (Alternating Direction Method of Multipliers) | Разделение задачи с учётом двух компонент регуляризации | Средняя скорость, отличная устойчивость | Да (в распределённых системах) | Да | `scikit-learn`: нет; альтернативы: `scikit-learn-extra`, `pyADMM`, `OSQP`, `CVXPY` | Гибкий; позволяет разделять L1 и L2-обновления | В промышленных масштабируемых системах | Позволяет **раздельно обрабатывать L1 и L2** компоненты, что невозможно в чистом Lasso |
| Subgradient Method | Использование субградиента для L1-части при наличии L2-штрафа | Низкая скорость, удовлетворительная устойчивость | Да (но медленно) | Да | `scikit-learn`: нет реализации; альтернативы: ручная реализация на `NumPy`, `PyTorch`, `TensorFlow` | Прост, но неэффективен из-за медленной сходимости | Только для учебных целей | Субградиент — от L1, но **полный градиент включает L2**, что делает метод сложнее, чем в Lasso |

### Итак

Выбор метода регуляризации и алгоритма решения зависит от размера данных, структуры признаков и целей модели. Ridge подходит для устойчивости при мультиколлинеарности и имеет аналитические решатели, Lasso обеспечивает отбор признаков, но чувствителен к корреляциям, а ElasticNet объединяет преимущества обоих, балансируя между разреженностью и устойчивостью. Для малых данных предпочтительны аналитические методы (SVD, QR, Холецкого), а для больших — итерационные (SAG, SAGA, Coordinate Descent, SGD), требующие масштабирования признаков, но обеспечивающие масштабируемость и гибкость.

In [9]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Данные: один признак
X_single = df_single[['x']] 
y_single = df_single['y']

# Модель
model = LinearRegression() # аналитический метод, матричное разложение SVD под капотом по умолчанию
model.fit(X_single, y_single)

# Результат
m_sk = model.coef_[0]    # наклон
b_sk = model.intercept_  # свободный член

print(f"\nМетод 3 (LinearRegression):")
print(f"y = {m_sk:.3f}x + {b_sk:.3f}")


Метод 3 (LinearRegression):
y = 2.972x + 0.427


## **Пример для ДВУХ признаков через матричную формулу:**

In [6]:
data_multi = {
    'x1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'x2': [2, 3, 3, 4, 4, 5, 5, 5, 5, 5],
    'y':  [4.0, 7.0, 9.5, 12.5, 15.0, 18.0, 21.0, 24.0, 26.5, 29.0]
}

df_multi = pd.DataFrame(data_multi)
print("\nДатасет (два признака):")
print(df_multi)


Датасет (два признака):
   x1  x2     y
0   1   2   4.0
1   2   3   7.0
2   3   3   9.5
3   4   4  12.5
4   5   4  15.0
5   6   5  18.0
6   7   5  21.0
7   8   5  24.0
8   9   5  26.5
9  10   5  29.0


In [7]:
n = len(df_multi)

# Матрица X: [1, x1, x2]
X_multi = np.column_stack([np.ones(n), df_multi['x1'], df_multi['x2']])
y_multi = df_multi['y'].values

# Решение: w = (X^T X)^{-1} X^T y
w_multi = np.linalg.inv(X_multi.T @ X_multi) @ X_multi.T @ y_multi

b_multi, w1, w2 = w_multi[0], w_multi[1], w_multi[2]

print(f"\nМножественная регрессия (матричная формула):")
print(f"y = {w1:.3f}·x1 + {w2:.3f}·x2 + {b_multi:.3f}")


Множественная регрессия (матричная формула):
y = 2.752·x1 + 0.154·x2 + 0.885


## **Пример для ДВУХ признаков из "коробки":**

In [8]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Данные
X_multi = df_multi[['x1', 'x2']]  # признаки
y_multi = df_multi['y']           # целевая переменная

# Модель "из коробки"
model = LinearRegression()
model.fit(X_multi, y_multi)

# Получаем коэффициенты
w1_sk = model.coef_[0]   # коэффициент при x1
w2_sk = model.coef_[1]   # коэффициент при x2
b_sk = model.intercept_  # свободный член

print(f"\nМножественная регрессия (LinearRegression):")
print(f"y = {w1_sk:.3f}·x1 + {w2_sk:.3f}·x2 + {b_sk:.3f}")


Множественная регрессия (LinearRegression):
y = 2.752·x1 + 0.154·x2 + 0.885


### В `LinearRegression` из `scikit-learn` нельзя напрямую выбрать метод решения (например, SVD или QR).  

Однако по умолчанию используется SVD-подход через LAPACK-решатель `gelsd`, который:

- Устойчив к вырожденным и плохо обусловленным матрицам,
- Корректно работает при $ n < p $ (больше признаков, чем объектов),
- Обеспечивает высокую численную стабильность.

Под капотом: вызывается `scipy.linalg.lstsq` с `lapack_driver='gelsd'` — метод на основе сингулярного разложения с divide-and-conquer.


### LAPACK-решатели, используемые для МНК

Вот три основных драйвера LAPACK, доступных в `scipy.linalg.lstsq`, которые могут использоваться для решения задачи $\min_{\mathbf{w}} \|\mathbf{X}\mathbf{w} - \mathbf{y}\|^2$:

| Метод   | Основа         | Устойчивость       | Скорость     | Когда используется |
|--------|----------------|--------------------|--------------|----------------------|
| `gelsd` | SVD + divide-and-conquer | Очень высокая | Средняя / высокая | По умолчанию в `scikit-learn` и `scipy` |
| `gelss` | Классическое SVD | Высокая         | Медленная  | Устаревшие системы, ограниченная память |
| `gelsy` | QR с поворотом по столбцам | Средняя (требует полного ранга) | Высокая | Если матрица $\mathbf{X}$ имеет полный ранг и важна скорость |


`LinearRegression` всегда использует `gelsd`, если не указано иное — это делает его наиболее универсальным и надёжным выбором для реальных данных.


### Почему нельзя выбрать метод вручную?

В отличие от `Ridge` или `LogisticRegression`, у `LinearRegression` нет параметра `solver`.  
Это сделано по следующим причинам:

1. **Универсальность**  
   SVD (`gelsd`) работает во всех случаях — даже если матрица $\mathbf{X}$ вырождена, имеет мультиколлинеарность или число признаков превышает число объектов.

2. **Надёжность "из коробки"**  
   `scikit-learn` стремится к простоте и предсказуемости: пользователь должен получать устойчивый результат без необходимости настраивать низкоуровневые параметры.

3. **Минимизация ошибок пользователя**  
   Если бы был выбор между SVD и QR, пользователь мог бы выбрать QR на плохих данных — получить нестабильные веса или не понять, почему модель ведёт себя странно.

4. **Производительность vs стабильность**  
   QR быстрее на хорошо обусловленных данных.  
   SVD (`gelsd`) — оптимальный компромисс между скоростью и устойчивостью для большинства реальных задач.


### QR-разложение напрямую можно реализовать через `scipy`:

```python
from scipy.linalg import qr, solve_triangular

Q, R = qr(X, mode='economic')
w = solve_triangular(R, Q.T @ y)  # w = R^{-1} Q^T y